In [1]:
%matplotlib inline

In [43]:
from keras_utilities import *
from keras_utilities.models.vgg16 import Vgg16
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import os

In [48]:
# Constants
PROD = False

if not PROD:
    dataset = 'sample'
else:
    dataset = 'train'

# Moving Data Around

In [3]:
%cd ~/Fisheries/

/home/ubuntu/Fisheries


In [4]:
%ls

analysis.ipynb                  sample_submission_stg2.csv.zip  test_stg2.7z
sample/                         test_stg1/                      train/
sample_submission_stg1.csv.zip  test_stg1.zip                   train.zip


In [5]:
%ls train/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [6]:
# ! rm -R /home/ubuntu/Fisheries/sample/

In [7]:
# create_data_sample('train', 'sample')

In [8]:
%ls train/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [9]:
%ls sample/

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [10]:
! ls -l train/ALB/ | wc -l

1719


In [11]:
! ls -l sample/ALB | wc -l 

427


We're good on counts, and seem to have successfully partitioned out 25% of the data to represent a training sample. Next, we have to work on coming up with a good validation set. Do the training set and validation sets come from different boats? If so, we might want to think about splitting out a few of the training boats to serve as validation boats.

Maybe a new way to handle validation set creation is to allow two types of partitioning:

1. A random percentage of the training data
2. Some subset of the training data that meets a certain criteria (we could start with something simply based on the file's name)

For cases where individual samples are totally iid we can use method 1, whereas if there is some relationship between examples (like some fish coming from the same boat, in this case) we can use method 2.

In [15]:
# TODO: a function for recursively creating a validation set from a training directory
# for subdir in os.listdir('train'):
#     train_path = 'train/' + subdir + '/'
#     val_path = 'val' + '/' + subdir + '/'
#     move_data_subset(train_path, val_path, subset_pct=.2, method='move')

In [16]:
! ls -l val/ALB | wc -l

348


In [17]:
! ls -l train/ALB | wc -l

1372


# Extracting Data

In [19]:
gen = ImageDataGenerator()

In [49]:
train_gen = gen.flow_from_directory(dataset, 
                                    target_size = (224,224), 
                                    batch_size = 32, 
                                    class_mode = 'categorical')

Found 960 images belonging to 8 classes.


In [21]:
val_gen = gen.flow_from_directory('val',
                                  target_size = (224,224),
                                  batch_size = 32,
                                  class_mode = 'categorical')

Found 784 images belonging to 8 classes.


In [38]:
mdl = Vgg16()

In [45]:
mdl.model.pop()
for layer in mdl.model.layers: layer.trainable = False
mdl.model.add(Dense(8, activation='softmax'))
mdl.compile()

In [50]:
mdl.model.fit_generator(train_gen, 
                        samples_per_epoch = train_gen.nb_sample, 
                        nb_epoch = 1, 
                        validation_data = val_gen, 
                        nb_val_samples=val_gen.nb_sample)

Epoch 1/1
960/960 [==============================] - 965s - loss: 3.8568 - acc: 0.5437 - val_loss: 2.7484 - val_acc: 0.6480
